#LIDC GDRO: Looking at Clustered Subgroups

Thomas Zeng

## Notebook Setup

Setup autoreload extension

In [ ]:
%load_ext autoreload
%autoreload 2

Clone repo (if on colab)

In [ ]:
# !git clone https://github.com/mtzig/LIDC_GDRO.git
# %cd /content/LIDC_GDRO

import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from dataloaders import InfiniteDataLoader
from datasets import NoduleDataset, SubclassedNoduleDataset
from models import TransferModel
from loss import ERMLoss, GDROLossAlt#, GDROLoss
from train import train, test
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
# import torchvision
from torchvision import transforms
import torch
from image_data_utils import getImages

Get available Device

In [ ]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

## ERM Model: Training ERM model to extract CNN features

First we load in the training data

In [ ]:
train_data, cv_data, test_data = getImages(split=True, sublabels=False, binary=True, device=DEVICE)

tr = SubclassedNoduleDataset(*train_data)
cv = SubclassedNoduleDataset(*cv_data)
tst = SubclassedNoduleDataset(*test_data)
tr_loader = InfiniteDataLoader(tr, batch_size=512)
cv_loader = InfiniteDataLoader(cv, len(cv))
tst_loader = InfiniteDataLoader(tst, len(tst))